# Notebook For Our Model

## Imports

In [1]:
import numpy as np
import math
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision.io import read_image, ImageReadMode
from torchvision.transforms import ToTensor
import glob
import matplotlib.pyplot as plot
from sklearn.metrics import confusion_matrix, accuracy_score
import cv2
import shutil
import csv
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

## Database

In [2]:
class XRAY(Dataset):
    def __init__(self, metadata, transform=None, target_transform=None):
        self.mode = 'Train'

        self.train_indexes = []
        self.test_indexes = []

        self.meta = metadata
        self.transform = transform
        self.target_transform = target_transform

        self.train_val_split()

    def train_val_split(self, val_size = 0.2):
        all_indexes = [i for i in range(len(self.meta))]
        self.train_indexes, self.test_indexes = train_test_split(all_indexes, test_size=val_size, random_state=42)

    def test(self):
        self.mode = 'Test'

    def train(self):
        self.mode = 'Train'

    def __len__(self):
        if self.mode == 'Train':
            return len(self.train_indexes)
        else:
            return len(self.test_indexes)
    
    def __getitem__(self, index):
        if self.mode == 'Train':
            index = self.train_indexes[index]
        else:
            index = self.test_indexes[index]

        img_path = self.meta[index][0]
        image = read_image(img_path, mode=ImageReadMode.GRAY).numpy()
        image = image.astype(np.float32)
        label = np.float32(self.meta[index][1])
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label


### Read Meta Data from CSV File and Create Dataset

In [3]:
metadataPath = "./MetaData.csv"
metaData = []

with open(rf"{metadataPath}") as metadataFile:
    csv_reader = csv.reader(metadataFile)

    for row in csv_reader:
        if (row[1] == "Healthy"):
            metaData.append([row[0], 0])
        else:
            metaData.append([row[0], 1])


xrays = XRAY(metaData)

### Define our Model

In [4]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cnn_model = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels = 6, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=5),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=5),
            nn.Conv2d(in_channels=16, out_channels=30, kernel_size=5),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2, stride=5)
        )

        self.fc_model = nn.Sequential(
            nn.Linear(in_features=1470, out_features=500),
            nn.Tanh(),
            nn.Linear(in_features=500, out_features=100),
            nn.Tanh(),
            nn.Linear(in_features=100, out_features=1)
        )
    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        x = F.sigmoid(x)

        return x


In [5]:
device = torch.device('cuda:0')
model = Model().to(device)

In [6]:
dataloader = DataLoader(xrays, batch_size=100, shuffle=False)

In [7]:
model.eval()
outputs = []
y_true = []

maxBatches = 10
currentBatch = 0

with torch.no_grad():
    for D in dataloader:
        if (currentBatch >= maxBatches):
            break
        image = D[0].to(device)
        label = D[1].to(device)
        
        y_hat = model(image)

        outputs.append(y_hat.cpu().detach().numpy())
        y_true.append(label.cpu().detach().numpy())
        currentBatch += 1
outputs = np.concatenate(outputs, axis = 0).squeeze()
y_true = np.concatenate(y_true, axis=0).squeeze()

In [9]:
def threshold(scores, threshold_val = 0.5, minimum = 0, maximum = 1):
    x = np.array(list(scores))

    x[x >= threshold_val] = maximum
    x[x < threshold_val] = minimum
    return x

In [10]:
accuracy_score(y_true, threshold(outputs))

0.57

In [13]:
eta = 0.0001
EPOCH = 50
optimizer = torch.optim.Adam(model.parameters(), lr=eta)
dataloader = DataLoader(xrays, batch_size = 32, shuffle=True)
model.train()

Model(
  (cnn_model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): AvgPool2d(kernel_size=2, stride=5, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): AvgPool2d(kernel_size=2, stride=5, padding=0)
    (6): Conv2d(16, 30, kernel_size=(5, 5), stride=(1, 1))
    (7): Tanh()
    (8): AvgPool2d(kernel_size=2, stride=5, padding=0)
  )
  (fc_model): Sequential(
    (0): Linear(in_features=1470, out_features=500, bias=True)
    (1): Tanh()
    (2): Linear(in_features=500, out_features=100, bias=True)
    (3): Tanh()
    (4): Linear(in_features=100, out_features=1, bias=True)
  )
)

In [14]:
for epoch in range(0, EPOCH):
    losses = []
    
    maxBatches = 20
    currentBatch = 0
    for D in dataloader:
        if (currentBatch >= maxBatches):
            break
        optimizer.zero_grad()
        image = D[0].to(device)
        label = D[1].to(device)
        y_hat = model(image)
        
        error = nn.BCELoss()
        loss = torch.sum(error(y_hat.squeeze(), label))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        currentBatch += 1
    if (epoch + 1) % 10 == 0:
        print(f"Train Epoch: {epoch + 1}\tLoss{np.mean(losses)}")

Train Epoch: 10	Loss0.6511446177959442
Train Epoch: 20	Loss0.6366436362266541
Train Epoch: 30	Loss0.6461820572614669
Train Epoch: 40	Loss0.67201127409935
Train Epoch: 50	Loss0.6566040188074111
